In [1]:
import math
from scipy.optimize import fsolve

# Antoine Equation

In [2]:
import math

class Component:
    """
    Represents a pure component within the flash mixture.
    """
    def __init__(self, A, B, C, temperature=None, vap_pressure=None):
        # Antoine equation constants
        self.A = A
        self.B = B
        self.C = C
        # use Antoine to calculate p_vap or T based on the inputs of the Component
        self.temperature = temperature if temperature else self.antoine_equation(vap_pressure=vap_pressure)
        self.vap_pressure = vap_pressure if vap_pressure else self.antoine_equation(temperature=temperature)

    def antoine_equation(self, temperature=None, vap_pressure=None):
        """
        Antoine equation to calculate temperature or vapour pressure of the pure component.

        Args:
            temperature (float): temperature of the pure component in K
            vap_pressure (float): vapour pressure of the pure component in mmHg
        Returns:
            float: temperature in K or vapour pressure in mmHg depending on method input
        """
        if temperature:
            return math.exp(self.A - self.B / (self.C + temperature))
        return self.B /(self.A - math.log(vap_pressure)) - self.C


def init_components(composition, temperature=None, vap_pressure=None):
    """
    Create a new Component object for each item in the composition list.

    Args:
        composition (list): list of lists containing component Antoine coefficients and feed amount
        temperature (float): temperature of the pure component in K
        vap_pressure (float): vapour pressure of the pure component in mmHg
    Returns:
        list[Component]: list of pure Component objects in the mixture
    """
    components = []
    for component in composition:
      A, B, C = component
      components.append(Component(A=A, B=B, C=C, temperature=temperature, vap_pressure=vap_pressure))
    return components

# pure component: [A, B, C]
COMPONENT_NAMES = ['benzene', 'toluene', 'o-xylene']
COMPOSITION = [[15.9008, 2788.51, -52.34],
               [16.0137, 3096.52, -53.67],
               [16.1156, 3395.57, -59.44]
]

# vapour pressure at T=385 K
specify_temp = init_components(COMPOSITION,temperature=385)
for i in range(len(specify_temp)):
  print(f'{COMPONENT_NAMES[i]}: {specify_temp[i].vap_pressure}')

print('\n')

# temperature at Pv=359.1 mmHg
specify_p_vap = init_components(COMPOSITION, vap_pressure=359.1)
for i in range(len(specify_p_vap)):
  print(f'{COMPONENT_NAMES[i]}: {specify_p_vap[i].temperature}')

benzene: 1841.5031116120479
toluene: 786.7984966323942
o-xylene: 294.61337098492055


benzene: 330.7122248610357
toluene: 359.3451932838054
o-xylene: 391.29792602433486


# Relative Volatility

In [3]:
import math

class Component:
    """
    Represents a pure component within the flash mixture.
    """
    def __init__(self, A, B, C, xk, pressure, temperature=None, vap_pressure=None):
        # Antoine equation constants
        self.A = A
        self.B = B
        self.C = C
        # liquid phase mole fraction
        self.xk = xk
        # use Antoine to calculate p_vap or T based on the inputs of the Component
        self.temperature = temperature if temperature else self.antoine_equation(vap_pressure=vap_pressure)
        self.vap_pressure = vap_pressure if vap_pressure else self.antoine_equation(temperature=temperature)
        # K factor
        self.k_factor = self.vap_pressure / pressure

    def antoine_equation(self, temperature=None, vap_pressure=None):
        """
        Antoine equation to calculate temperature or vapour pressure of the pure component.

        Args:
            temperature (float): temperature of the pure component in K
            vap_pressure (float): vapour pressure of the pure component in mmHg
        Returns:
            float: temperature in K or vapour pressure in mmHg depending on method input
        """
        if temperature:
            return math.exp(self.A - self.B / (self.C + temperature))
        return self.B /(self.A - math.log(vap_pressure)) - self.C

    def set_relative_volatility(self, key_k_factor):
        """
        Calculate the relative volitility with the k factor of the key component.

        Args:
            key_k_factor (float): K factor of the key component
        """
        self.relative_volatility = self.k_factor / key_k_factor


class Average_Volatility:
    """
    Object to calculate the average volatility of a mixture.
    """
    def __init__(self, composition, pressure, temperature, key_component):
        self.pressure = pressure
        self.temperature = temperature
        self.key_component = key_component
        self.components = self.init_components(composition)

    def init_components(self, composition):
        """
        Create a new Component object for each item in the composition list.

        Args:
            composition (list): list of lists containing component Antoine coefficients and liquid mole fraction
        Returns:
            list[Component]: list of pure Component objects in the mixture
        """
        components = []
        for component in composition:
          A, B, C, xk = component
          components.append(Component(A=A, B=B, C=C, xk=xk, pressure=self.pressure, temperature=self.temperature))
        return components

    def get_average_volatility(self):
        """
        Calculates average volatility of the mixture.

        Parses the K factor of the key component to calculate the relative volatility
        for each component. Sums the product of the liquid phase mole fraction (x) and
        the relative volatility of each component.

        Returns:
            average_volatility (float): average volatility of the mixture
            components (list[Components]): list of pure Component objects in the mixture
        """
        key_k_factor = self.components[self.key_component].k_factor
        average_volatility = 0
        for component in self.components:
            component.set_relative_volatility(key_k_factor)
            average_volatility += component.xk * component.relative_volatility
        return average_volatility, self.components


COMPONENT_NAMES = ['benzene', 'toluene', 'o-xylene']
# pure component: [A, B, C, xk]
COMPOSITION = [[15.9008, 2788.51, -52.34, 0.1013],
               [16.0137, 3096.52, -53.67, 0.3469],
               [16.1156, 3395.57, -59.44, 0.5518]
]

# pressure in mmHg, temperature in K, key component corresponds to list index of COMPOSITION
test_avg_vol = Average_Volatility(COMPOSITION, 750, 385, 1)
avg_vol, components = test_avg_vol.get_average_volatility()
print(f'average volatility: {avg_vol}')
for i in range(len(components)):
    print(f'{COMPONENT_NAMES[i]} relative volatility: {components[i].relative_volatility}')


average volatility: 0.7906119857371698
benzene relative volatility: 2.340501563607371
toluene relative volatility: 1.0
o-xylene relative volatility: 0.37444577264179624


# Bubble Point Calculation

In [4]:
import math
from scipy.optimize import fsolve

class Component:
    """
    Represents a pure component within the flash mixture.
    """
    def __init__(self, A, B, C, fk, temperature=None, vap_pressure=None):
        # Antoine equation constants
        self.A = A
        self.B = B
        self.C = C
        # feed molar flow
        self.fk = fk
        # use Antoine to calculate p_vap or T based on the inputs of the Component
        self.temperature = temperature if temperature else self.antoine_equation(vap_pressure=vap_pressure)
        self.vap_pressure = vap_pressure if vap_pressure else self.antoine_equation(temperature=temperature)

    def antoine_equation(self, temperature=None, vap_pressure=None):
        """
        Antoine equation to calculate temperature or vapour pressure of the pure component.

        Args:
            temperature (float): temperature of the pure component in K
            vap_pressure (float): vapour pressure of the pure component in mmHg
        Returns:
            float: temperature in K or vapour pressure in mmHg depending on method input
        """
        if temperature:
            return math.exp(self.A - self.B / (self.C + temperature))
        return self.B /(self.A - math.log(vap_pressure)) - self.C

    def set_relative_volatility(self, key_vap_pressure):
        """
        Calculate the relative volitility with the vapour pressure of the key component.

        Args:
            key_vap_pressure (float): Vapour pressure of the key component
        """
        self.relative_volatility = self.vap_pressure / key_vap_pressure

class Bubble_Point:
    """
    Calculator for bubble point temperature if pressure is passed and
    bubble point pressure if temperature is passed.
    """
    def __init__(self, composition):
        self.key_split_fraction = 0   # all split fractions are 0 for bubble point
        self.composition = composition
        key_component = 0
        # find total feed flow and check for most abundant species
        self.total_feed = 0
        for i in range(len(composition)):
            self.total_feed += composition[i][-1]
            if composition[i][-1] > composition[key_component][-1]:
                key_component = i
        # for bubble point, n = k'
        self.key_component = key_component
        self.k_prime = key_component

    def specify_temperature(self, temperature):
        """
        Run flash calculations at a specified temperature to find bubble point pressure.

        Args:
            temperature (float): Mixture temperature in K
        """
        self.flash_intermediate_steps(temperature)
        self.calc_pressure = self.get_calc_pressure()

    def specify_pressure(self, pressure, t_guess):
        """
        Run flash calculations at a specified pressure to find bubble point temperature.

        Iterate on bubble point temperature with fsolve based on an initial guess.

        Args:
            pressure (float): Mixture pressure in mmHg
            t_guess (float): Inital guess for bubble point temperature in K
        """
        self.pressure = pressure
        self.calc_temperature = fsolve(self.temperature_iteration, t_guess)[0]

    def temperature_iteration(self, temperature):
        """
        Iterative function for fsolve to optimize that runs flash calculations.

        Args:
            temperature (float): Mixture temperature in K

        Returns:
            float: The difference between guessed and calculated bubble point temperature.
        """
        self.flash_intermediate_steps(temperature)
        return temperature - self.get_calc_temperature(self.pressure)

    def flash_intermediate_steps(self, temperature):
        """
        Run flash calculation algorithm with helper functions.

        1. Initialize components at a given temperature
        2. Calculate relative volatility for each component
        3. Calculate the average volatility

        Args:
            temperature (float): Mixture temperture in K
        """
        self.components = self.init_components(self.composition, temperature)
        self.get_relative_volatility()
        average_volatility_helper = lambda component: component.relative_volatility * component.fk / self.total_feed
        self.average_volatility = sum(map(average_volatility_helper,self.components))

    def init_components(self, composition, temperature):
        """
        Create a new Component object for each item in the composition list.

        Args:
            composition (list): list of lists containing component Antoine coefficients and feed amount
            temperature (float): temperature of the mixture in K
        Returns:
            list[Component]: list of pure Component objects in the mixture
        """
        components = []
        for component in composition:
            A, B, C, fk = component
            components.append(Component(A=A, B=B, C=C, fk=fk, temperature=temperature))
        return components

    def get_relative_volatility(self):
        """
        Calculate the relative volatility of each component with the vapour pressure
        of the key component.
        """
        key_vap_pressure = self.components[self.key_component].vap_pressure
        for component in self.components:
            component.set_relative_volatility(key_vap_pressure)

    def get_calc_temperature(self, pressure):
        """
        Calculate the bubble point temperature with the bubble point equation.

        Args:
            pressure (float): Mixture pressure in mmHg
        Returns:
            float: Bubble point temperature in K from the antoine equation of the K prime component
        """
        k_prime_component = self.components[self.k_prime]
        p_vap_k_prime = k_prime_component.relative_volatility * pressure / self.average_volatility
        return k_prime_component.antoine_equation(vap_pressure=p_vap_k_prime)

    def get_calc_pressure(self):
        """
        Calculate the bubble point pressure with the bubble point equation.

        Returns:
            float: Bubble point pressure in mmHg
        """
        k_prime_component = self.components[self.k_prime]
        return  self.average_volatility * k_prime_component.vap_pressure / k_prime_component.relative_volatility

COMPONENT_NAMES = ['benzene', 'toluene', 'o-xylene']
# pure component: [A, B, C, fk]
COMPOSITION = [[15.9008, 2788.51, -52.34, 30],
               [16.0137, 3096.52, -53.67, 50],
               [16.1156, 3395.57, -59.44, 40]
]

# bubble point temperature at P = 750 mmHg and T_guess = 385 K
test_case_bubble = Bubble_Point(composition=COMPOSITION)
test_case_bubble.specify_pressure(pressure=750, t_guess=385)
print(test_case_bubble.calc_temperature)

# bubble point pressure at T = 378.93598 K
test_case_bubble.specify_temperature(378.93598)
print(test_case_bubble.calc_pressure)

378.93598707293114
749.9998511740287


# Dew Point

In [5]:
import math
from scipy.optimize import fsolve

class Component:
    """
    Represents a pure component within the flash mixture.
    """
    def __init__(self, A, B, C, fk, temperature=None, vap_pressure=None):
        # Antoine equation constants
        self.A = A
        self.B = B
        self.C = C
        # feed molar flow
        self.fk = fk
        # use Antoine to calculate p_vap or T based on the inputs of the Component
        self.temperature = temperature if temperature else self.antoine_equation(vap_pressure=vap_pressure)
        self.vap_pressure = vap_pressure if vap_pressure else self.antoine_equation(temperature=temperature)

    def antoine_equation(self, temperature=None, vap_pressure=None):
        """
        Antoine equation to calculate temperature or vapour pressure of the pure component.

        Args:
            temperature (float): temperature of the pure component in K
            vap_pressure (float): vapour pressure of the pure component in mmHg
        Returns:
            float: temperature in K or vapour pressure in mmHg depending on method input
        """
        if temperature:
            return math.exp(self.A - self.B / (self.C + temperature))
        return self.B /(self.A - math.log(vap_pressure)) - self.C

    def set_relative_volatility(self, key_vap_pressure):
        """
        Calculate the relative volitility with the vapour pressure of the key component.

        Args:
            key_vap_pressure (float): Vapour pressure of the key component
        """
        self.relative_volatility = self.vap_pressure / key_vap_pressure

class Dew_Point:
    """
    Calculator for dew point temperature if pressure is passed and
    dew point pressure if temperature is passed.
    """
    def __init__(self, composition):
        self.key_split_fraction = 1   # all split fractions are 1 for dew point
        self.composition = composition
        key_component = 0
        # find total feed flow and check for most abundant species
        self.total_feed = 0
        for i in range(len(composition)):
            self.total_feed += composition[i][-1]
            if composition[i][-1] > composition[key_component][-1]:
                key_component = i
        # for dew point, n = k'
        self.key_component = key_component
        self.k_prime = key_component

    def specify_temperature(self, temperature):
        """
        Run flash calculations at a specified temperature to find dew point pressure.

        Args:
            temperature (float): Mixture temperature in K
        """
        self.flash_intermediate_steps(temperature)
        self.calc_pressure = self.get_calc_pressure()

    def specify_pressure(self, pressure, t_guess):
        """
        Run flash calculations at a specified pressure to find dew point temperature.

        Iterate on dew point temperature with fsolve based on an initial guess.

        Args:
            pressure (float): Mixture pressure in mmHg
            t_guess (float): Inital guess for dew point temperature in K
        """
        self.pressure = pressure
        self.calc_temperature = fsolve(self.temperature_iteration, t_guess)[0]

    def temperature_iteration(self, temperature):
        """
        Iterative function for fsolve to optimize that runs flash calculations.

        Args:
            temperature (float): Mixture temperature in K

        Returns:
            float: The difference between guessed and calculated dew point temperature.
        """
        self.flash_intermediate_steps(temperature)
        return temperature - self.get_calc_temperature(self.pressure)

    def flash_intermediate_steps(self, temperature):
        """
        Run flash calculation algorithm with helper functions.

        1. Initialize components at a given temperature
        2. Calculate relative volatility for each component
        3. Calculate Kn (zk / relative_volatility)

        Args:
            temperature (float): Mixture temperture in K
        """
        self.components = self.init_components(self.composition, temperature)
        self.get_relative_volatility()
        # sum zk / alpha_k/n for each component
        kn_helper = lambda component: (component.fk / self.total_feed) / component.relative_volatility
        self.kn = sum(map(kn_helper,self.components))

    def init_components(self, composition, temperature):
        """
        Create a new Component object for each item in the composition list.

        Args:
            composition (list): list of lists containing component Antoine coefficients and feed amount
            temperature (float): temperature of the mixture in K
        Returns:
            list[Component]: list of pure Component objects in the mixture
        """
        components = []
        for component in composition:
            A, B, C, fk = component
            components.append(Component(A=A, B=B, C=C, fk=fk, temperature=temperature))
        return components

    def get_relative_volatility(self):
        """
        Calculate the relative volatility of each component with the vapour pressure
        of the key component.
        """
        key_vap_pressure = self.components[self.key_component].vap_pressure
        for component in self.components:
            component.set_relative_volatility(key_vap_pressure)

    def get_calc_temperature(self, pressure):
        """
        Calculate the dew point temperature with the dew point equation.

        Args:
            pressure (float): Mixture pressure in mmHg
        Returns:
            float: Dew point temperature in K from the antoine equation of the K prime component
        """
        k_prime_component = self.components[self.k_prime]
        p_vap_k_prime =  pressure * self.kn
        return k_prime_component.antoine_equation(vap_pressure=p_vap_k_prime)

    def get_calc_pressure(self):
        """
        Calculate the dew point pressure with the dew point equation.

        Returns:
            float: Dew point pressure in mmHg
        """
        k_prime_component = self.components[self.k_prime]
        return  k_prime_component.vap_pressure / self.kn

COMPONENT_NAMES = ['benzene', 'toluene', 'o-xylene']
# pure component: [A, B, C, fk]
COMPOSITION = [[15.9008, 2788.51, -52.34, 30],
               [16.0137, 3096.52, -53.67, 50],
               [16.1156, 3395.57, -59.44, 40]
]

# dew point temperature at P = 750 mmHg and T_guess = 385 K
test_case_dew = Dew_Point(composition=COMPOSITION)
test_case_dew.specify_pressure(pressure=750, t_guess=385)
print(test_case_dew.calc_temperature)

# dew point pressure at T = 395.127254 K
test_case_dew.specify_temperature(395.127254)
print(test_case_dew.calc_pressure)

395.1272546886555
749.999985255126


# Flash Case 1
### split fraction and temperature/pressure specified

In [6]:
import math
from scipy.optimize import fsolve

class Component:
    """
    Represents a pure component within the flash mixture.
    """
    def __init__(self, A, B, C, fk, temperature=None, vap_pressure=None):
        # Antoine equation constants
        self.A = A
        self.B = B
        self.C = C
        # feed molar flow
        self.fk = fk
        # use Antoine to calculate p_vap or T based on the inputs of the Component
        self.temperature = temperature if temperature else self.antoine_equation(vap_pressure=vap_pressure)
        self.vap_pressure = vap_pressure if vap_pressure else self.antoine_equation(temperature=temperature)

    def antoine_equation(self, temperature=None, vap_pressure=None):
        """
        Antoine equation to calculate temperature or vapour pressure of the pure component.

        Args:
            temperature (float): temperature of the pure component in K
            vap_pressure (float): vapour pressure of the pure component in mmHg
        Returns:
            float: temperature in K or vapour pressure in mmHg depending on method input
        """
        if temperature:
            return math.exp(self.A - self.B / (self.C + temperature))
        return self.B /(self.A - math.log(vap_pressure)) - self.C

    def set_relative_volatility(self, key_vap_pressure):
        """
        Calculate the relative volitility with the vapour pressure of the key component.

        Args:
            key_vap_pressure (float): Vapour pressure of the key component
        """
        self.relative_volatility = self.vap_pressure / key_vap_pressure

    def set_phase_flows(self, key_split_fraction):
        """
        Calculate the split fraction, liquid phase molar flow, and vapour phase molar flow.

        Args:
            key_split_fraction (float): Split fraction of the key component
        """
        self.split_fraction = self.relative_volatility * key_split_fraction / ((self.relative_volatility - 1)* key_split_fraction + 1)
        self.vapour_flow = self.split_fraction * self.fk
        self.liquid_flow = (1 - self.split_fraction) * self.fk

    def set_phase_composition(self, total_liquid, total_vapour):
        """
        Calculate the liquid phase mole fraction and vapour phase mole fraction.

        Args:
            total_liquid (float): Total liquid phase molar flow of the mixture
            total_vapour (float): Total vapour phase molar flow of the mixture
        """
        self.liquid_fraction = self.liquid_flow / total_liquid
        self.vapour_fraction = self.vapour_flow / total_vapour

class Flash_Case_1:
    """
    Case 1 flash calculator: key component split fraction and temperature/pressure specified.
    """
    def __init__(self, composition, key_component, key_split_fraction):
        self.key_component = key_component
        self.key_split_fraction = key_split_fraction
        self.composition = composition

    def specify_temperature(self, temperature):
        """
        Run flash calculations at a specified temperature.

        Args:
            temperature (float): Mixture temperature in K
        """
        self.flash_intermediate_steps(temperature)
        self.calc_pressure = self.get_calc_pressure()

    def specify_pressure(self, pressure, t_guess):
        """
        Run flash calculations at a specified pressure.

        Iterate on flash temperature with fsolve based on an initial guess.

        Args:
            pressure (float): Mixture pressure in mmHg
            t_guess (float): Inital guess for temperature in K
        """
        self.pressure = pressure
        self.calc_temperature = fsolve(self.temperature_iteration, t_guess)[0]

    def temperature_iteration(self, temperature):
        """
        Iterative function for fsolve to optimize that runs flash calculations.

        Args:
            temperature (float): Mixture temperature in K

        Returns:
            float: The difference between guessed and calculated temperature
        """
        self.flash_intermediate_steps(temperature)
        return temperature - self.get_calc_temperature(self.pressure)

    def flash_intermediate_steps(self, temperature):
        """
        Run flash calculation algorithm with helper functions.

        1. Initialize components at a given temperature
        2. Calculate relative volatility for each component
        3. Calculate the molar flows in the liquid and vapour phases for each component
        4. Calculate the mole fractions in the liquid and vapour phases for each component
        5. Calculate the average volatility

        Args:
            temperature (float): Mixture temperture in K
        """
        self.components = self.init_components(self.composition,temperature)
        self.get_relative_volatility()
        self.get_phase_flows()
        self.get_phase_composition()
        average_volatility_helper = lambda component: component.relative_volatility * component.liquid_fraction
        self.average_volatility = sum(map(average_volatility_helper,self.components))

    def init_components(self, composition, temperature):
        """
        Create a new Component object for each item in the composition list.

        Args:
            composition (list): list of lists containing component Antoine coefficients and feed amount
            temperature (float): temperature of the mixture in K
        Returns:
            list[Component]: list of pure Component objects in the mixture
        """
        components = []
        for component in composition:
            A, B, C, fk = component
            components.append(Component(A=A, B=B, C=C, fk=fk, temperature=temperature))
        return components

    def get_relative_volatility(self):
        """
        Calculate the relative volatility of each component with the vapour pressure
        of the key component.
        """
        key_vap_pressure = self.components[self.key_component].vap_pressure
        for component in self.components:
            component.set_relative_volatility(key_vap_pressure)

    def get_phase_flows(self):
        """
        Calculate the vapour and liquid flows of the mixture and each component.
        Determine the K prime component.
        """
        self.liquid_flow = 0
        self.vapour_flow = 0
        curr_k_prime = 0
        for i in range(len(self.components)):
            component = self.components[i]
            component.set_phase_flows(self.key_split_fraction)
            self.liquid_flow += component.liquid_flow
            self.vapour_flow += component.vapour_flow
            # check for k prime, most abundant species in the liquid phase
            if component.liquid_flow > self.components[curr_k_prime].liquid_flow:
                curr_k_prime = i
        self.k_prime = curr_k_prime

    def get_phase_composition(self):
        """
        Calculate the vapour and liquid mole fractions of each component.
        """
        for component in self.components:
            component.set_phase_composition(self.liquid_flow, self.vapour_flow)

    def get_calc_temperature(self, pressure):
        """
        Calculate the flash temperature with the bubble point equation.

        Args:
            pressure (float): Mixture pressure in mmHg
        Returns:
            float: Flash temperature in K from the antoine equation of the K prime component
        """
        k_prime_component = self.components[self.k_prime]
        p_vap_k_prime = k_prime_component.relative_volatility * pressure / self.average_volatility
        return k_prime_component.antoine_equation(vap_pressure=p_vap_k_prime)

    def get_calc_pressure(self):
        """
        Calculate the flash pressure with the bubble point equation.

        Returns:
            float: Flash pressure in mmHg
        """
        k_prime_component = self.components[self.k_prime]
        return  self.average_volatility * k_prime_component.vap_pressure / k_prime_component.relative_volatility

COMPONENT_NAMES = ['benzene', 'toluene', 'o-xylene']
# pure component: [A, B, C, fk]
COMPOSITION = [[15.9008, 2788.51, -52.34, 30],
               [16.0137, 3096.52, -53.67, 50],
               [16.1156, 3395.57, -59.44, 40]
]


# case 1 flash, toluene key component, key split fraction = 0.8
test_case_1 = Flash_Case_1(composition=COMPOSITION, key_component=1, key_split_fraction=0.8)

# case 1 flash at P = 750 mmHg, T_guess = 385 K
test_case_1.specify_pressure(pressure=750, t_guess=385)
print(test_case_1.calc_temperature)
for component in test_case_1.components:
    print(f'{component.split_fraction}')

print('\n')

# case 1 flash at T = 391.6 K
test_case_1.specify_temperature(391.6)
print(test_case_1.calc_pressure)
for component in test_case_1.components:
    print(f'{component.split_fraction}')

391.58978211698604
0.9017864981293741
0.8
0.6055542529007784


750.2174127854468
0.9017838842101058
0.8
0.6055631876761653


# Flash Case 2
### temperature and pressure specified

In [7]:
import math
from scipy.optimize import fsolve

class Component:
    """
    Represents a pure component within the flash mixture.
    """
    def __init__(self, A, B, C, fk, temperature=None, vap_pressure=None):
        # Antoine equation constants
        self.A = A
        self.B = B
        self.C = C
        # feed molar flow
        self.fk = fk
        # use Antoine to calculate p_vap or T based on the inputs of the Component
        self.temperature = temperature if temperature else self.antoine_equation(vap_pressure=vap_pressure)
        self.vap_pressure = vap_pressure if vap_pressure else self.antoine_equation(temperature=temperature)

    def antoine_equation(self, temperature=None, vap_pressure=None):
        """
        Antoine equation to calculate temperature or vapour pressure of the pure component.

        Args:
            temperature (float): temperature of the pure component in K
            vap_pressure (float): vapour pressure of the pure component in mmHg
        Returns:
            float: temperature in K or vapour pressure in mmHg depending on method input
        """
        if temperature:
            return math.exp(self.A - self.B / (self.C + temperature))
        return self.B /(self.A - math.log(vap_pressure)) - self.C

    def set_relative_volatility(self, key_vap_pressure):
        """
        Calculate the relative volitility with the vapour pressure of the key component.

        Args:
            key_vap_pressure (float): Vapour pressure of the key component
        """
        self.relative_volatility = self.vap_pressure / key_vap_pressure

    def set_phase_flows(self, key_split_fraction):
        """
        Calculate the split fraction, liquid phase molar flow, and vapour phase molar flow.

        Args:
            key_split_fraction (float): Split fraction of the key component
        """
        self.split_fraction = self.relative_volatility * key_split_fraction / ((self.relative_volatility - 1)* key_split_fraction + 1)
        self.vapour_flow = self.split_fraction * self.fk
        self.liquid_flow = (1 - self.split_fraction) * self.fk

    def set_phase_composition(self, total_liquid, total_vapour):
        """
        Calculate the liquid phase mole fraction and vapour phase mole fraction.

        Args:
            total_liquid (float): Total liquid phase molar flow of the mixture
            total_vapour (float): Total vapour phase molar flow of the mixture
        """
        self.liquid_fraction = self.liquid_flow / total_liquid
        self.vapour_fraction = self.vapour_flow / total_vapour

class Flash_Case_2:
    """
    Case 2 flash calculator: temperature and pressure specified.
    """
    def __init__(self, composition, key_component, temperature, pressure):
        self.key_component = key_component
        self.composition = composition
        self.temperature = temperature
        self.pressure = pressure

    def specify_split_fraction(self, split_fraction_guess):
        """
        Run flash calculations based on specified temperature and pressure.

        Iterate on split fraction with fsolve based on an initial guess.

        Args:
            split_fraction_guess (float): Initial guess for key component split fraction
        """
        self.key_split_fraction = fsolve(self.split_fraction_iteration, split_fraction_guess)[0]

    def split_fraction_iteration(self, split_fraction):
        """
        Iterative function for fsolve to optimize that runs flash calculations.

        Args:
            split_fraction (float): Split fraction of the key component

        Returns:
            float: The difference between guessed and calculated split fraction of the key component
        """
        self.key_split_fraction = split_fraction[0]
        self.flash_intermediate_steps(self.temperature)
        return split_fraction - self.get_calc_split_fraction()

    def flash_intermediate_steps(self, temperature):
        """
        Run flash calculation algorithm with helper functions.

        1. Initialize components at a given temperature
        2. Calculate relative volatility for each component
        3. Calculate the molar flows in the liquid and vapour phases for each component
        4. Calculate the mole fractions in the liquid and vapour phases for each component
        5. Calculate the average volatility

        Args:
            temperature (float): Mixture temperture in K
        """
        self.components = self.init_components(self.composition, temperature)
        self.get_relative_volatility()
        self.get_phase_flows()
        self.get_phase_composition()
        average_volatility_helper = lambda component: component.relative_volatility * component.liquid_fraction
        self.average_volatility = sum(map(average_volatility_helper,self.components))

    def init_components(self, composition, temperature):
        """
        Create a new Component object for each item in the composition list.

        Args:
            composition (list): list of lists containing component Antoine coefficients and feed amount
            temperature (float): temperature of the mixture in K
        Returns:
            list[Component]: list of pure Component objects in the mixture
        """
        components = []
        for component in composition:
            A, B, C, fk = component
            components.append(Component(A=A, B=B, C=C, fk=fk, temperature=temperature))
        return components

    def get_relative_volatility(self):
        """
        Calculate the relative volatility of each component with the vapour pressure
        of the key component.
        """
        key_vap_pressure = self.components[self.key_component].vap_pressure
        for component in self.components:
            component.set_relative_volatility(key_vap_pressure)

    def get_phase_flows(self):
        """
        Calculate the vapour and liquid flows of the mixture and each component.
        Determine the K prime component.
        """
        self.liquid_flow = 0
        self.vapour_flow = 0
        curr_k_prime = 0
        for i in range(len(self.components)):
            component = self.components[i]
            component.set_phase_flows(self.key_split_fraction)
            self.liquid_flow += component.liquid_flow
            self.vapour_flow += component.vapour_flow
            if component.liquid_flow > self.components[curr_k_prime].liquid_flow:
                curr_k_prime = i
        self.k_prime = curr_k_prime

    def get_phase_composition(self):
        """
        Calculate the vapour and liquid mole fractions of each component.
        """
        for component in self.components:
            component.set_phase_composition(self.liquid_flow, self.vapour_flow)

    def get_calc_split_fraction(self):
        """
        Calculate the key split fraction with the theta and phi equations.

        Returns:
            float: Split fraction of the key component
        """
        phi = self.vapour_flow / (self.liquid_flow + self.vapour_flow)
        key_k_factor = self.components[self.key_component].vap_pressure / self.pressure
        theta = key_k_factor * phi / (1 - phi)
        return theta / (1 + theta)


COMPONENT_NAMES = ['benzene', 'toluene', 'o-xylene']
# pure component: [A, B, C, fk]
COMPOSITION = [[15.9008, 2788.51, -52.34, 30],
               [16.0137, 3096.52, -53.67, 50],
               [16.1156, 3395.57, -59.44, 40]
]

# case 2 flash, toluene key component, T = 385 K, P = 750 mmHg
test_case_2 = Flash_Case_2(composition=COMPOSITION, key_component=1, temperature=385, pressure=750)

# intial key split fraction guess = 0.4
test_case_2.specify_split_fraction(0.4)
print(test_case_2.key_split_fraction)
for component in test_case_2.components:
    print(f'{component.split_fraction}')

0.37070252310311397
0.5796071914020121
0.37070252310311397
0.1807147562162519


# Flash Case 3
### phi and temperature/pressure specified

In [8]:
import math
from scipy.optimize import fsolve

class Component:
    """
    Represents a pure component within the flash mixture.
    """
    def __init__(self, A, B, C, fk, temperature=None, vap_pressure=None):
        # Antoine equation constants
        self.A = A
        self.B = B
        self.C = C
        # feed molar flow
        self.fk = fk
        # use Antoine to calculate p_vap or T based on the inputs of the Component
        self.temperature = temperature if temperature else self.antoine_equation(vap_pressure=vap_pressure)
        self.vap_pressure = vap_pressure if vap_pressure else self.antoine_equation(temperature=temperature)

    def antoine_equation(self, temperature=None, vap_pressure=None):
        """
        Antoine equation to calculate temperature or vapour pressure of the pure component.

        Args:
            temperature (float): temperature of the pure component in K
            vap_pressure (float): vapour pressure of the pure component in mmHg
        Returns:
            float: temperature in K or vapour pressure in mmHg depending on method input
        """
        if temperature:
            return math.exp(self.A - self.B / (self.C + temperature))
        return self.B /(self.A - math.log(vap_pressure)) - self.C

    def set_relative_volatility(self, key_vap_pressure):
        """
        Calculate the relative volitility with the vapour pressure of the key component.

        Args:
            key_vap_pressure (float): Vapour pressure of the key component
        """
        self.relative_volatility = self.vap_pressure / key_vap_pressure

    def set_phase_flows(self, key_split_fraction):
        """
        Calculate the split fraction, liquid phase molar flow, and vapour phase molar flow.

        Args:
            key_split_fraction (float): Split fraction of the key component
        """
        self.split_fraction = self.relative_volatility * key_split_fraction / ((self.relative_volatility - 1)* key_split_fraction + 1)
        self.vapour_flow = self.split_fraction * self.fk
        self.liquid_flow = (1 - self.split_fraction) * self.fk

    def set_phase_composition(self, total_liquid, total_vapour):
        """
        Calculate the liquid phase mole fraction and vapour phase mole fraction.

        Args:
            total_liquid (float): Total liquid phase molar flow of the mixture
            total_vapour (float): Total vapour phase molar flow of the mixture
        """
        self.liquid_fraction = self.liquid_flow / total_liquid
        self.vapour_fraction = self.vapour_flow / total_vapour

class Flash_Case_3:
    """
    Case 3 flash calculator: feed vapourization (phi) and temperature/pressure specified.
    """
    def __init__(self, composition, key_component, phi):
        self.key_component = key_component
        self.composition = composition
        # feed vapourization
        self.phi = phi

    def specify_temperature(self, temperature, p_guess):
        """
        Run flash calculations at a specified temperature.

        Iterate on flash pressure with fsolve based on an initial guess.

        Args:
            temperature (float): Mixture temperature in K
            p_guess (float): Initial guess for pressure in mmHg
        """
        self.temperature = temperature
        self.calc_pressure = fsolve(self.pressure_iteration, p_guess)[0]

    def pressure_iteration(self, pressure):
        """
        Iterative function for fsolve to optimize pressure with flash calculations.

        Args:
            pressure (float): Mixture pressure in mmHg

        Returns:
            float: The difference between guessed and calculated pressure
        """
        self.pressure = pressure[0]
        self.flash_intermediate_steps(self.temperature)
        return pressure - self.get_calc_pressure()

    def specify_pressure(self, pressure, t_guess):
        """
        Run flash calculations at a specified pressure.

        Iterate on flash temperature with fsolve based on an initial guess.

        Args:
            pressure (float): Mixture pressure in mmHg
            t_guess (float): Inital guess for temperature in K
        """
        self.pressure = pressure
        self.calc_temperature = fsolve(self.temperature_iteration, t_guess)[0]

    def temperature_iteration(self, temperature):
        """
        Iterative function for fsolve to optimize temperature with flash calculations.

        Args:
            temperature (float): Mixture temperature in K

        Returns:
            float: The difference between guessed and calculated temperature
        """
        self.flash_intermediate_steps(temperature)
        return temperature - self.get_calc_temperature(self.pressure)

    def flash_intermediate_steps(self, temperature):
        """
        Run flash calculation algorithm with helper functions.

        1. Initialize components at a given temperature
        2. Calculate key split fraction
        3. Calculate relative volatility for each component
        4. Calculate the molar flows in the liquid and vapour phases for each component
        5. Calculate the mole fractions in the liquid and vapour phases for each component
        6. Calculate the average volatility

        Args:
            temperature (float): Mixture temperture in K
        """
        self.components = self.init_components(self.composition,temperature)
        self.key_split_fraction = self.get_key_split_fraction()
        self.get_relative_volatility()
        self.get_phase_flows()
        self.get_phase_composition()
        average_volatility_helper = lambda component: component.relative_volatility * component.liquid_fraction
        self.average_volatility = sum(map(average_volatility_helper,self.components))

    def init_components(self, composition, temperature):
        """
        Create a new Component object for each item in the composition list.

        Args:
            composition (list): list of lists containing component Antoine coefficients and feed amount
            temperature (float): temperature of the mixture in K
        Returns:
            list[Component]: list of pure Component objects in the mixture
        """
        components = []
        for component in composition:
            A, B, C, fk = component
            components.append(Component(A=A, B=B, C=C, fk=fk, temperature=temperature))
        return components

    def get_relative_volatility(self):
        """
        Calculate the relative volatility of each component with the vapour pressure
        of the key component.
        """
        key_vap_pressure = self.components[self.key_component].vap_pressure
        for component in self.components:
            component.set_relative_volatility(key_vap_pressure)

    def get_phase_flows(self):
        """
        Calculate the vapour and liquid flows of the mixture and each component.
        Determine the K prime component.
        """
        self.liquid_flow = 0
        self.vapour_flow = 0
        curr_k_prime = 0
        for i in range(len(self.components)):
            component = self.components[i]
            component.set_phase_flows(self.key_split_fraction)
            self.liquid_flow += component.liquid_flow
            self.vapour_flow += component.vapour_flow
            if component.liquid_flow > self.components[curr_k_prime].liquid_flow:
                curr_k_prime = i
        self.k_prime = curr_k_prime

    def get_phase_composition(self):
        """
        Calculate the vapour and liquid mole fractions of each component.
        """
        for component in self.components:
            component.set_phase_composition(self.liquid_flow, self.vapour_flow)

    def get_calc_temperature(self, pressure):
        """
        Calculate the flash temperature with the bubble point equation.

        Args:
            pressure (float): Mixture pressure in mmHg
        Returns:
            float: Flash temperature in K from the antoine equation of the K prime component
        """
        k_prime_component = self.components[self.k_prime]
        p_vap_k_prime = k_prime_component.relative_volatility * pressure / self.average_volatility
        return k_prime_component.antoine_equation(vap_pressure=p_vap_k_prime)

    def get_calc_pressure(self):
        """
        Calculate the flash pressure with the bubble point equation.

        Returns:
            float: Flash pressure in mmHg
        """
        k_prime_component = self.components[self.k_prime]
        return  self.average_volatility * k_prime_component.vap_pressure / k_prime_component.relative_volatility

    def get_key_split_fraction(self):
        """
        Calculate the key split fraction with the theta and phi equations.

        Returns:
            float: Split fraction of the key component
        """
        key_k_factor = self.components[self.key_component].vap_pressure / self.pressure
        theta = key_k_factor * self.phi / (1 - self.phi)
        return theta / (1 + theta)

COMPONENT_NAMES = ['benzene', 'toluene', 'o-xylene']
# pure component: [A, B, C, fk]
COMPOSITION = [[15.9008, 2788.51, -52.34, 30],
               [16.0137, 3096.52, -53.67, 50],
               [16.1156, 3395.57, -59.44, 40]
]

# case 3 flash, toluene key component, phi = 0.5
test_case_3 = Flash_Case_3(composition=COMPOSITION, key_component=1, phi=0.5)

# case 3 flash at P = 750 mmHg, T_guess = 385 K
test_case_3.specify_pressure(pressure=750, t_guess=385)
print(test_case_3.calc_temperature)
for component in test_case_3.components:
    print(f'{component.split_fraction}')

print('\n')

# case 3 flash at  T = 387.36, P_guess = 750
test_case_3.specify_temperature(387.3600622278725, 750)
print(test_case_3.calc_pressure)
for component in test_case_3.components:
    print(f'{component.split_fraction}')


387.3600622278725
0.7225838637691463
0.5284686086316366
0.29747634138359386


749.9999999999992
0.7225838637691465
0.5284686086316369
0.29747634138359413


# Ethylene-to-Ethanol Process

### Unit Conversion for Constants

In [23]:
import numpy as np

E2E_PRESSURE = 68.5 * 750.06    # convert pressure to mmHg
E2E_COMPONENT_NAMES = ["methane", "ethylene", "propylene", "diethyl-ether", "ethanol", "isopropanol", "water"]
# pure component: [A, B, C, fk]
E2E_COMPOSITION = np.array([[6.61184, 389.93, 266.0, 200],
                        [6.74756, 585.0, 255.0, 1198.77],
                        [6.81960, 785.0, 247.0, 266.71],
                        [7.4021, 1391.4, 273.16, 2.421],
                        [8.04494, 1554.3, 222.65, 90.79],
                        [6.66040, 813.055, 132.96, 1.8802],
                        [8.10765, 1750.286, 235.0, 680.72]])
E2E_KEY_COMPONENT = 3

# convert A and B from log10 to ln
E2E_COMPOSITION[:,:2] *= np.log(10)
# convert C from Celsius to Kelvin
E2E_COMPOSITION[:, 2] -= 273.15
E2E_COMPOSITION



array([[ 1.52243242e+01,  8.97847005e+02, -7.15000000e+00,
         2.00000000e+02],
       [ 1.55368311e+01,  1.34701228e+03, -1.81500000e+01,
         1.19877000e+03],
       [ 1.57027093e+01,  1.80752930e+03, -2.61500000e+01,
         2.66710000e+02],
       [ 1.70439651e+01,  3.20381690e+03,  1.00000000e-02,
         2.42100000e+00],
       [ 1.85241589e+01,  3.57890801e+03, -5.05000000e+01,
         9.07900000e+01],
       [ 1.53361378e+01,  1.87212832e+03, -1.40190000e+02,
         1.88020000e+00],
       [ 1.86685540e+01,  4.03018245e+03, -3.81500000e+01,
         6.80720000e+02]])

### Bubble Point Temperature

In [20]:
e2e_bubble_point = Bubble_Point(E2E_COMPOSITION)
e2e_bubble_point.specify_pressure(pressure=E2E_PRESSURE, t_guess=590)
print(e2e_bubble_point.calc_temperature)

318.7201089796104


### Dew Point Temperature

In [36]:
e2e_dew_point = Dew_Point(E2E_COMPOSITION)
e2e_dew_point.specify_pressure(pressure=E2E_PRESSURE, t_guess=480)
print(e2e_dew_point.calc_temperature)

491.37930967826765


### Case 1 Flash

In [29]:
DEE_SPLIT_FRACTION = 0.5
e2e_flash = Flash_Case_1(composition=E2E_COMPOSITION, key_component=E2E_KEY_COMPONENT, key_split_fraction=DEE_SPLIT_FRACTION)
e2e_flash.specify_pressure(pressure=E2E_PRESSURE, t_guess=590)
print(e2e_flash.calc_temperature)
for component in e2e_flash.components:
    print(f'Split Fraction: {component.split_fraction}, xk: {component.liquid_fraction}, yk: {component.vapour_fraction}')

451.4371949600338
25.952249979037738
11.95100107051527
4.505847907324579
1.0
0.7051167336453309
0.5347362150737828
0.3569459016638147


### Upper Recycle Flash Condition

In [25]:
UPPER_DEE_SPLIT_FRACTION = 0.75
e2e_flash_upper = Flash_Case_1(composition=E2E_COMPOSITION, key_component=E2E_KEY_COMPONENT, key_split_fraction=UPPER_DEE_SPLIT_FRACTION)
e2e_flash_upper.specify_pressure(pressure=E2E_PRESSURE, t_guess=590)
print(e2e_flash_upper.calc_temperature)
for component in e2e_flash_upper.components:
    print(f'Split Fraction: {component.split_fraction}, xk: {component.liquid_fraction}, yk: {component.vapour_fraction}')

474.20467939662495
Split Fraction: 0.9838992774227147, xk: 0.008240175305599774, yk: 0.09596651992386325
Split Fraction: 0.9675135268675492, xk: 0.09965509776305675, yk: 0.5656294591468757
Split Fraction: 0.922673073426833, xk: 0.052775351806278466, yk: 0.12001243704042444
Split Fraction: 0.75, xk: 0.0015488019496560869, yk: 0.0008855134491418616
Split Fraction: 0.7085081650822536, xk: 0.0677213331139372, yk: 0.03137053927179395
Split Fraction: 0.6322552585510524, xk: 0.0017693412723907235, yk: 0.0005797431251073533
Split Fraction: 0.5589421825311678, xk: 0.768289898789081, yk: 0.18555578804279352


In [26]:
LOWER_DEE_SPLIT_FRACTION = 0.25
e2e_flash_lower = Flash_Case_1(composition=E2E_COMPOSITION, key_component=E2E_KEY_COMPONENT, key_split_fraction=LOWER_DEE_SPLIT_FRACTION)
e2e_flash_lower.specify_pressure(pressure=E2E_PRESSURE, t_guess=590)
print(e2e_flash_lower.calc_temperature)
for component in e2e_flash_lower.components:
    print(f'Split Fraction: {component.split_fraction}, xk: {component.liquid_fraction}, yk: {component.vapour_fraction}')

418.2237656177065
Split Fraction: 0.9281055187088431, xk: 0.014372276201061832, yk: 0.12882923600856178
Split Fraction: 0.8439206004094297, xk: 0.18701715931353327, yk: 0.702141325480135
Split Fraction: 0.6480103719388217, xk: 0.09383593165078752, yk: 0.1199522330095194
Split Fraction: 0.25, xk: 0.001814914027016417, yk: 0.00042007020496256356
Split Fraction: 0.1556880187438331, xk: 0.07661979270614008, yk: 0.009810254827476378
Split Fraction: 0.13244095362348218, xk: 0.0016304335201321307, yk: 0.00017282773092726554
Split Fraction: 0.08185855874515276, xk: 0.6247094925813288, yk: 0.038674052738417425
